## Sohbet Modelleri Kullanımı

In [1]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

GPT 3.5 turbo kullanan bir ChatOpenAI objesi oluşturuyoruz

In [ ]:
llm = ChatOpenAI(temperature=0, openai_api_key="")
llm

Burada istem şablonu oluşturuyoruz

In [4]:
prompt = PromptTemplate.from_template("What are the popular puppet plays in {country} culture?")

In [5]:
prompt

PromptTemplate(input_variables=['country'], output_parser=None, partial_variables={}, template='What are the popular puppet plays in {country} culture?', template_format='f-string', validate_template=True)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)
res = chain.run("German")

In [7]:
print(res)

Some popular puppet plays in German culture include:

1. Kasperl and Seppel: This is a classic puppet play that features two characters, Kasperl and Seppel, who go on various adventures and solve problems.

2. Punch and Judy: This is a traditional puppet play that originated in England but is also popular in Germany. It features the characters of Punch and Judy, who engage in slapstick comedy and often end up in violent confrontations.

3. The Three Little Pigs: This is a popular children's story that has been adapted into a puppet play in Germany. It tells the story of three pigs who build houses of straw, sticks, and bricks, and their encounter with a hungry wolf.

4. The Pied Piper of Hamelin: This is another popular children's story that has been adapted into a puppet play in Germany. It tells the story of a piper who is hired to rid a town of its rats, but when the townspeople refuse to pay him, he leads their children away.

5. The Magic Flute: This is a puppet play based on the 

In [8]:
type(res)

str

Şimdi bize verilen çıktıyı "Çıktı Ayrıştırıcıları" kullanarak formatlayacağız.

In [9]:
template = """\
For the following text, extract the following information:

name: Extrtact the name of the puppet plays? \
If this information is not found, output -1.

content: Extract any information that defines the content of listed puppet plays?\
If this information is not found, output -1.

Format the output as JSON with the following keys:
name
content

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\nname: Extrtact the name of the puppet plays? If this information is not found, output -1.\n\ncontent: Extract any information that defines the content of listed puppet plays?If this information is not found, output -1.\n\nFormat the output as JSON with the following keys:\nname\ncontent\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [10]:
formatted_res = prompt_template.format_messages(text=res)
response = llm(formatted_res)
print(response.content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


{
  "name": ["Kasperl and Seppel", "Punch and Judy", "The Three Little Pigs", "The Pied Piper of Hamelin", "The Magic Flute"],
  "content": ["classic puppet play featuring two characters who go on various adventures and solve problems", "traditional puppet play featuring characters who engage in slapstick comedy and often end up in violent confrontations", "children's story about three pigs who build houses and encounter a hungry wolf", "children's story about a piper who leads away the town's children", "puppet play based on the famous opera by Mozart about a prince who must rescue his love from the Queen of the Night with the help of a magic flute"]
}


In [11]:
type(response.content)

str

Büyük dil modelinin vermiş olduğu "string" tipindeki çıktımızı Python "dictionary" tipine çevireceğiz

In [12]:
#gerekli paketler
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [13]:
name_schema = ResponseSchema(name="name",
                             description="Extrtact the name of the puppet plays? \
If this information is not found, output -1.")
content_schema = ResponseSchema(name="content",
                                      description="Extract any information that defines the content of listed puppet plays?\
If this information is not found, output -1.")

response_schemas = [name_schema, content_schema]

In [14]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
#format_instructions = output_parser.get_format_instructions()
#print(format_instructions)

In [15]:
output_dict = output_parser.parse(response.content)
print(output_dict)

{'name': ['Kasperl and Seppel', 'Punch and Judy', 'The Three Little Pigs', 'The Pied Piper of Hamelin', 'The Magic Flute'], 'content': ['classic puppet play featuring two characters who go on various adventures and solve problems', 'traditional puppet play featuring characters who engage in slapstick comedy and often end up in violent confrontations', "children's story about three pigs who build houses and encounter a hungry wolf", "children's story about a piper who leads away the town's children", 'puppet play based on the famous opera by Mozart about a prince who must rescue his love from the Queen of the Night with the help of a magic flute']}


In [16]:
type(output_dict)

dict

In [20]:
print("All names: " , output_dict.get('name'))
print("--------------------------------------")
print("First element: " , output_dict.get('name')[0])

All names:  ['Kasperl and Seppel', 'Punch and Judy', 'The Three Little Pigs', 'The Pied Piper of Hamelin', 'The Magic Flute']
--------------------------------------
First element:  Kasperl and Seppel


##### LangChain şemalarının kullanımı

In [35]:
from langchain.schema import (
    HumanMessage,
    AIMessage
)

In [36]:
res2 = llm.predict_messages([HumanMessage(content="Ottoman tradition had a rich tradition of puppet plays. One of them known as Karagöz and Hacivat. Who are some of the other notable characters in the Karagöz and Hacivat plays?")])
res2

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


AIMessage(content='Some of the other notable characters in the Karagöz and Hacivat plays include:\n\n1. Tuzsuz Deli Bekir - a crazy character who is always looking for trouble\n2. Tiryaki - a drug addict who is always in a daze\n3. Civan - a strong and brave character who is always ready for a fight\n4. Zenne - a female character who is often portrayed as flirtatious and seductive\n5. Kavuklu - a wise and knowledgeable character who often provides advice and guidance to the other characters\n6. Pişekar - a cook who is always hungry and looking for food\n7. İbiş - a mischievous character who loves to play pranks on the other characters.', additional_kwargs={}, example=False)

Burada tekrar bir istemci şeması düzenleyerek çıktılarımızı daha okunaklı bir hale getiriyoruz

In [44]:
template2 = """\
For the following text, extract the following information:

character_name: Extrtact the name of the characters? \
If this information is not found, output -1.

role: Extract any information that defines characteristics of that character?\
If this information is not found, output -1.

Format the output as JSON with the following keys:
character_name
role

text: {text}
"""

prompt_template2 = ChatPromptTemplate.from_template(template2)
print(prompt_template2)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ncharacter_name: Extrtact the name of the characters? If this information is not found, output -1.\n\nrole: Extract any information that defines characteristics of that character?If this information is not found, output -1.\n\nFormat the output as JSON with the following keys:\ncharacter_name\nrole\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [46]:
formatted_res2 = prompt_template2.format_messages(text=res2)
response2 = llm(formatted_res2)
print(response2.content)

[
  {
    "character_name": "Tuzsuz Deli Bekir",
    "role": "a crazy character who is always looking for trouble"
  },
  {
    "character_name": "Tiryaki",
    "role": "a drug addict who is always in a daze"
  },
  {
    "character_name": "Civan",
    "role": "a strong and brave character who is always ready for a fight"
  },
  {
    "character_name": "Zenne",
    "role": "a female character who is often portrayed as flirtatious and seductive"
  },
  {
    "character_name": "Kavuklu",
    "role": "a wise and knowledgeable character who often provides advice and guidance to the other characters"
  },
  {
    "character_name": "Pişekar",
    "role": "a cook who is always hungry and looking for food"
  },
  {
    "character_name": "İbiş",
    "role": "a mischievous character who loves to play pranks on the other characters"
  }
]
